In [1]:
from rfid_app import db
from rfid_app.models import Student, Bus, Route, TripHistory, BusStop, bus_and_routes, bus_stops_and_routes
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [2]:
engine = create_engine('postgresql://localhost:5432/rfid_app')
Session = sessionmaker(bind=engine)
session = Session()

/Users/john_jensen/coding/rfid_user_tracker/ENV/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
s = Student.query.filter_by(id=1).first()

In [4]:
conn_str='postgresql://localhost:5432/rfid_app'

In [5]:
stu_df = pd.read_sql('select * from students', con=conn_str)

In [6]:
stu_df

,id,first_name,last_name,home_stop_id,school_stop,present
0,1,sue,test,4,None,False
1,2,bob,boblast,1,None,False
2,3,tom,cat,5,None,False
3,4,sara,omaley,2,None,False
4,5,steve,mcqueen,3,None,False


In [7]:
students = session.query(Student).all()
buses = session.query(Bus).all()
routes = session.query(Route).all()
bus_stops = session.query(BusStop).all()
trip_history = session.query(TripHistory).all()
bus_routes = session.query(bus_and_routes).all() # (route_id, bus_id)
bus_stops_routes = session.query(bus_stops_and_routes).all() # (stop_id, route_id)

In [33]:
main_data = {
    'Students': ['{} {}'.format(student.first_name, student.last_name) for student in students],
    'Home Stop': [student.home_stop_id for student in students],
#     'Route':[],
#     'Bus #': []
}
pd.DataFrame(main_data)


,Home Stop,Students
0,4,sue test
1,1,bob boblast
2,5,tom cat
3,2,sara omaley
4,3,steve mcqueen


In [18]:
stu_df = pd.read_sql('select * from students', con=conn_str)
stu_df

,id,first_name,last_name,home_stop_id,school_stop,present
0,1,sue,test,4,None,False
1,2,bob,boblast,1,None,False
2,3,tom,cat,5,None,False
3,4,sara,omaley,2,None,False
4,5,steve,mcqueen,3,None,False


In [20]:
# stu_routes = pd.read_sql('select')

In [25]:
bus_stops = pd.read_sql('select * from bus_stops', con=conn_str)
bus_stops

,id,name
0,1,mercer
1,2,greenlake
2,3,westlake
3,4,queenanne
4,5,caphill


In [110]:
bus_stop_to_routes = pd.read_sql('select name from routes where routes.id in \
                                (select route_id from bus_stop_route where (stop_id=4))', con=conn_str)
bus_stop_to_routes

,name
0,blue


In [116]:
bus_routes = pd.read_sql('select * from routes', con=conn_str)
bus_routes

,id,name
0,1,blue
1,2,orange
2,3,green


In [129]:
bus_id = pd.read_sql('select * from bus_route where (route_id=1)', con=conn_str)
bus_num = pd.read_sql('select * from buses where (id=2)', con=conn_str)
bus_num
bus_id

,route_id,bus_id
0,1,2


In [68]:
pd.DataFrame(bus_stops_routes)

,stop_id,route_id
0,1,1
1,2,3
2,3,3
3,4,1
4,5,2


In [125]:
pd.read_sql('select * from students, bus_stops where students.home_stop_id=bus_stops.id', con=conn_str)

,id,first_name,last_name,home_stop_id,school_stop,present,id,name
0,1,sue,test,4,None,False,4,queenanne
1,2,bob,boblast,1,None,False,1,mercer
2,3,tom,cat,5,None,False,5,caphill
3,4,sara,omaley,2,None,False,2,greenlake
4,5,steve,mcqueen,3,None,False,3,westlake


In [126]:
pd.read_sql('select * from students, bus_stop_route where students.home_stop_id=bus_stop_route.stop_id', con=conn_str)


,id,first_name,last_name,home_stop_id,school_stop,present,stop_id,route_id
0,2,bob,boblast,1,None,False,1,1
1,4,sara,omaley,2,None,False,2,3
2,5,steve,mcqueen,3,None,False,3,3
3,1,sue,test,4,None,False,4,1
4,3,tom,cat,5,None,False,5,2


In [122]:
pd.read_sql('select * from bus_stop_route, routes where bus_stop_route.route_id=routes.id', con=conn_str)

,stop_id,route_id,id,name
0,1,1,1,blue
1,2,3,3,green
2,3,3,3,green
3,4,1,1,blue
4,5,2,2,orange


In [131]:
pd.read_sql('select * from buses, bus_route where buses.id=bus_route.bus_id', con=conn_str)

,id,number,route_id,bus_id
0,2,45,1,2
1,1,321,3,1
2,3,99,2,3
